<a href="https://colab.research.google.com/github/dushime-benue/ai-class/blob/main/tech_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip install langchain-openai -q
!pip install langchain-community -q
!pip install langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.6 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
class ZhipuAI_embeddings:
    def __init__(self, model_name: str = 'embedding-3'):
        self.model_name = model_name
        self.base_url = "https://open.bigmodel.cn/api/paas/v4"
        self.embedding = self._init_model()
    def _init_model(self) -> OpenAIEmbeddings:
        return OpenAIEmbeddings(
            model=self.model_name,
            base_url=self.base_url,
            api_key=userdata.get("apikey")
        )
embeddings =ZhipuAI_embeddings().embedding

In [5]:
from langchain_openai import ChatOpenAI
client  = ChatOpenAI(
    base_url ="https://open.bigmodel.cn/api/paas/v4/",
    api_key = userdata.get("apikey"),
    model = "glm-4.5"
)

In [6]:
client.invoke("hello")

AIMessage(content='Hello! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 8, 'total_tokens': 21, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'glm-4.5', 'system_fingerprint': None, 'id': '20250912143236e2705d6260f84b46', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--00b81255-7b60-482d-abd6-9093da3c2ac2-0', usage_metadata={'input_tokens': 8, 'output_tokens': 13, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [7]:
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import PyPDFLoader
def doc_parsing(file_path) -> list[Document]:
    if file_path.endswith("pdf"):
        doc = PyPDFLoader(file_path=file_path)
        doc = doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        print(f" the number of chucks :{len(raw_chunks)}")
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    elif file_path.endswith(".txt"):
        doc = TextLoader(file_path=file_path)
        doc =doc.load()
        semantic_splitter = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=95
        )
        full_text = doc[0].page_content if doc else ""
        if not full_text:
            return []
        raw_chunks = semantic_splitter.split_text(full_text)
        docs = [Document(page_content=chunk, metadata=doc[0].metadata) for chunk in raw_chunks]
        return docs
    else:
       return []

In [8]:
new_template = """
Your name is Cyber Cruz, a smart, yet sassy assistant working at takenolab. You have a deep knowledge of takenolab operations.
Your task is to be supportive, provide proper guidance to students who are having trouble with the course content.
You must answer them directly and precisely. If you don't have any advice for them, kindly tell them so and do not generate any other response.

When you receive:
• chat_history: the previous turns of the conversation (if any)
• question: the student’s current problem
• context: optionally, the content of any uploaded documents relevant to the current question

If `context` is non-empty, you **must** use it to inform your answer. If you don’t have enough information
from the question and context combined, tell the student you can’t help further.
If `chat_history` is provided, use it to understand the full context of the current question.

<chat_history>
{chat_history}
</chat_history>

student problem:
{question}

uploaded document context (if any):
{context}

your smart advice or solution:

"""

In [11]:

from langchain_core.prompts import PromptTemplate
import gradio as gr
from langchain.chains import LLMChain
from langchain_core.documents import Document
from typing import TypedDict, List
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from typing import Optional,Tuple, Dict

vector_store = InMemoryVectorStore(embeddings)
def define_term(term: str="", context_docs:List[Optional[Document]]=None,chat_history: List[Tuple[str, str]]=None):
    prompt_template = PromptTemplate.from_template(template=new_template)
    chain = prompt_template | client
    history_str = ""
    if chat_history:
        for human_msg, ai_msg in chat_history:
            history_str += f"User: {human_msg}\n Assistant: {ai_msg}\n"
    context_text =""
    if context_docs:
         context_text = "\n\n".join(d.page_content for d in context_docs)
    response = chain.invoke({
        "question": term,
        "context": context_text,
        "chat_history": history_str
    })
    return response.content

def summarize_notes(text: str, num_queries: int =3)  -> List[str]:
    sub_query_prompt = PromptTemplate.from_template(template=SUB_QUERY_TEMPLATE)
    sub_query_chain = sub_query_prompt | client.client

    response = sub_query_chain.invoke({
        "question": text

    })

    queries = [q.strip() for q in response.content.split('-') if q.strip()]
    print(f"Generated sub-queries: {queries}")
    return queries[:num_queries]



def retrieve_and_answer_with_history(question: str, chat_history: List[Dict])->str:
    # retrieved = vector_store.similarity_search(question)
    formatted_chat_history_for_llm = []
    for msg in chat_history:
        if msg["role"] == "user":
            current_user_msg = msg["content"]
        elif msg["role"] == "assistant":
            formatted_chat_history_for_llm.append((current_user_msg, msg["content"]))
            current_user_msg = None
    if len(vector_store.store.items()) >= 0:
        transformed_queries = summarize_notes(question, num_queries=3)
        all_retrieved_docs = []
        seen_doc_contents = set()

        for query in transformed_queries:
            retrieved_for_query = vector_store.similarity_search(query)
            for doc in retrieved_for_query:
                if doc.page_content not in seen_doc_contents:
                    all_retrieved_docs.append(doc)
                    seen_doc_contents.add(doc.page_content)
        ai_response = define_term(question, context_docs=all_retrieved_docs, chat_history=formatted_chat_history_for_llm)
        return ai_response
    ai_response = define_term(question,chat_history=formatted_chat_history_for_llm)
    return ai_response
def doc_loader(file_path):
    docs = doc_parsing(file_path)
    if not docs:
        return "No content found or processed in the document."
    _ = vector_store.add_documents(documents=docs)
    return docs[0].page_content[:200] + "..."

def interface():
    iface = gr.ChatInterface(
        fn=retrieve_and_answer_with_history,
        chatbot=gr.Chatbot(height=200, type='messages', label="AI Assistant"),
        textbox=gr.Textbox(lines=2,submit_btn=True ),
        title="Takenolab AIClass Assistant (Conversational RAG)",
        type='messages',
        description="Ask a question about your course content and get smart advice, supporting multi-turn conversations.",
        streaming =True
    )
    docs_interface = gr.Interface(
        fn=doc_loader,
        inputs=gr.File(label="Choose a file to upload",
                       type='filepath',
                       file_count='single',
                       show_label=True
                       ),
        description="Upload a document to run retrieval‐augmented generation.",
        outputs=gr.TextArea()
    )
    table = gr.TabbedInterface(
        [iface,docs_interface],
        tab_names= ['Chat', "Upload File for RAG"],
        title="cyber cruz llm"
    )
    iface.launch(debug=True, server_port=3000)


In [12]:
def detect_emotion(text: str) -> str:
    text = text.lower()
    if any(word in text for word in ["confused", "lost", "don't get it"]):
        return "supportive"
    elif any(word in text for word in ["angry", "frustrated", "annoyed"]):
        return "calm"
    elif any(word in text for word in ["excited", "happy", "awesome"]):
        return "enthusiastic"
    return "neutral"

In [10]:
interface()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cb5a2502e9bb6c8767.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 667, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 349, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2274, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1779, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/utils.py", line 882, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:3000 <> https://cb5a2502e9bb6c8767.gradio.live
